In [1]:
import pandas as pd
from arrow import now

time_start = now()

def score(arg: str, ) -> float:
    result = 0
    result += 0.5 if len(arg) < 1798 else 0
    result += 0.5 if len(arg.split()) < 290 else 0
    return result

In [2]:
test_df = pd.read_csv(filepath_or_buffer='../input/llm-detect-ai-generated-text/test_essays.csv')
test_df['generated'] = test_df['text'].apply(func=score, )

In [3]:
test_df.drop(columns=['prompt_id', 'text']).to_csv(path_or_buf='submission.csv', index=False)